<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
# Initial Setup

<style>
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
</style>

## Mount drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Speech Folder Name

In [ ]:
snr = "0"
snr_str = str(snr).replace("-", "M")
snr_noise = str(snr).replace("MIX", "0")
snr_noise_str = snr_noise.replace("-", "M")
folder_name = f"TIMIT_{snr_str}_FINAL"

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Unzip file

In [ ]:
import zipfile


zip_file_path =\
 f"/content/drive/MyDrive/TDTU/voice-processing/{folder_name}.zip"
extract_path = '/content/'


try:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
  print(f"Successfully unzipped {zip_file_path} to {extract_path}")
except FileNotFoundError:
  print(f"Error: File not found at {zip_file_path}")
except zipfile.BadZipFile:
  print(f"Error: Invalid zip file at {zip_file_path}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

Successfully unzipped /content/drive/MyDrive/TDTU/voice-processing/TIMIT_0_FINAL.zip to /content/


In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import librosa
import pickle
import IPython.display as ipd
import time
import random
random.seed(1)

from re import I

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torch import nn

from sklearn.metrics import auc


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Environment Configuration

In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {} device".format(device))
# %% The variables in this cell can be customised
learning_rate = 100e-4 # Learning rate for the EB, FB and DB layers
learning_rate_DN = 1e-5 # Learning rate for the discriminative network
LR_factor = 0.7 # The factor with which to decrease the learning rate after each epoch
l2_weight = 1e-4

training_epochs = 10 # The number of epochs during training
concatenates = 10 # The number of files to concatenate
training_batch_size = 3  # The number of forward steps per backward step. Multiplied with "concatenates" this is the mini-batch size
testing_batch_size = 200 # The number of files in the testing split. Validation split is half of this
AN_weight = 0.1 # The scalar referred to as "alpha" in the paper
output_folder = "/content" # The name of the folder in which to store the results and models

"""Kernel sizes"""
k_EB1 = 55
k_EB2 = 160
k_EB3 = 160
k_EB4 = 160

k_FB = 160

k_DB1 = 0
k_DB2 = 0
k_DB3 = 0

k_DN1 = 55
k_DN2 = 15
k_DN3 = 5

# %%
training = 0 # Flag denoting whether the model is being trained or tested
validation = 0 # Flag denoting whether to use the testing or validation split

# Initialize the loss function
loss_primary = nn.BCELoss()
loss_secondary = nn.CrossEntropyLoss()

training_results_big = {
     "training" : [],
     "learning_rate" : [],
      "epochs":[],
     "loss_DB":[],
     "loss_AN":[],
     "test_TP":[],
     "test_FP":[],
     "test_TN":[],
     "test_FN":[],
     "time_passed" : []

    }

training_results_AUC = {
     "TP" : [],
     "FP" : [],
      "TN":[],
     "FN":[],
    }


noises_buffer = []
validation = 0



Using cuda device


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
# Utility Functions

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Get data from path

In [ ]:
def get_paths_data(train:int=0) -> list[list[str, str]]:

    """
     Parameters
    ----------
    train : int, optional
        Flag to indicate whether to get paths for training data (1) or testing data (0). The default is 0.

    Returns
    -------
    data : list
        The list of pairs of file paths. Each pair contains the path to the .wav file and the path to the corresponding label file.
        <br>
        The data list clipped to 2000 samples for faster processing.

    """

    # Đường dẫn đến thư mục dữ liệu
    base_path = f'/content/{folder_name}'
    data_folder = 'TRAIN' if train else 'TEST'
    data_folder_path = os.path.join(base_path, data_folder)

    # Danh sách để lưu trữ các cặp đường dẫn
    data = []

    # Duyệt qua từng thư mục từ dr1 đến dr8
    for dr in range(1, 9):
      dr_folder = os.path.join(data_folder_path, f'DR{dr}')

      # Tìm tất cả các thư mục con trong thư mục dr
      subfolders = glob.glob(os.path.join(dr_folder, '*'))

      for subfolder in subfolders:
          # Tìm tất cả các file .wav trong thư mục con
        wav_files = glob.glob(os.path.join(subfolder, '*.WAV'))
        for wav_file in wav_files:

          parts = wav_file.split("-")

          label_file = "";

          if(len(parts) == 1):
            label_file = wav_file\
            .replace(f'{folder_name}', f'{folder_name}/labels')[:-4]
          else:

            label_file = parts[0]\
            .replace(f'{folder_name}', f'{folder_name}/labels')

          if os.path.exists(label_file):
            data.append([wav_file, label_file])

    return data[:1000]

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Get element from data

In [ ]:
def get_element(idx: int, data: list[list[str, str]], train: int = 0) -> tuple[np.ndarray, np.ndarray, str, str]:
    """
    Used in __getitem__ of the dataloader class. Takes the data and label paths and return the
    data stored inside them

    Parameters
    ----------
    idx : int
        Index of the data to be unpackaged. Randomly generated by the dataloader class.
    data : list
        List containing both the path to the data file and its corresponding label file.

    Returns
    -------
    data_sequence : numpy array
        The raw speech data in 16 bit format.
    label_sequence : numpy array
        The corresponding labels matching the speech data.

    """

    data_path, label_path = data[idx]
    SNR = "-10"

    noise_type = 'CL'

    parts = data_path.split("-")

    if(len(parts) != 1):
      flag = 0
      if(len(parts) == 4):
        flag = 1

      type_noise = parts[2 + flag].split(".")[0]
      if(type_noise == 'BAB'):
        noise_type = 'N1'
      if(type_noise == 'FAC'):
        noise_type = 'N2'


    data_tensor = []
    label_tensor = []

    audio, sr = librosa.load(data_path, sr=None)

    data_tensor.append(audio)
    label_tensor.append(np.fromfile(label_path,sep="\n"))

    data_tensor[-1] = data_tensor[-1][0:len(label_tensor[-1])*160]
    x = np.zeros((1,1,sum(len(item) for item in data_tensor)))
    y = np.zeros((sum(len(item) for item in label_tensor)))
    x[0,0,:] = np.hstack(data_tensor)
    y = np.hstack(label_tensor)

    return x, y, noise_type, SNR


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
## Data Loader Classes

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Train Data Loader Class

In [ ]:
class TIMIT_train(Dataset):
    """
    Dataset class for the training data. Inherits from the torch.utils.data.Dataset class.
    The __getitem__ method is used to load the data and labels from the TIMIT dataset.
    """
    def __init__(self):
        self.data = get_paths_data(train=1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return get_element(idx, self.data, train=1)

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Test Data Loader Class

In [ ]:
class TIMIT_test(Dataset):

    """
    Dataset class for the testing data. Inherits from the torch.utils.data.Dataset class.
    The __getitem__ method is used to load the data and labels from the TIMIT dataset.
    """
    def __init__(self):
        self.data = get_paths_data(train=0)

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        """
        Function for unpacking the speech and label files and save them into numpy arrays
        """
        return get_element(idx, self.data, train=0)


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
</style>
# Model Definition

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1.2em;
        color: #465E5B;
    }
</style>
<span>
<strong>The architecture of the model is following the image below.</strong>
</span>
<div style="text-align:center;margin:auto;">
    <img src="./imgs/image.png" width="70%">
</div>

In [ ]:
class VAD_model(nn.Module):
    """
    The model class for the Voice Activity Detection model. Inherits from the torch.nn.Module class.
    Based on the model described in the paper "Adversarial Multi-Task Deep Neural Networks for Robust Voice Activity Detection".

    The model consists of the following layers:
    - 4 Encoder Blocks (EB)
    - 1 Framing Block (FB)
    - 3 Adversarial Network Blocks (AN)
    And some dropout layers and batch normalization layers.
    """
    def __init__(self):
        super(VAD_model, self).__init__()
        self.EB1 = nn.Conv1d(1,30, k_EB1,stride=1,padding='same')
        torch.nn.init.kaiming_uniform_(self.EB1.weight, a=0.01, mode='fan_in',nonlinearity='leaky_relu')
        self.relu1 = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.drop1 = nn.Dropout(p=0.1)

        self.EB2 = nn.Conv1d(30,15, k_EB2,stride=1,padding='same')
        torch.nn.init.kaiming_uniform_(self.EB2.weight, a=0.01, mode='fan_in',nonlinearity='leaky_relu')
        self.relu2 = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.drop2 = nn.Dropout(p=0.1)

        self.EB3 = nn.Conv1d(15,7, k_EB3,stride=1,padding='same')
        torch.nn.init.kaiming_uniform_(self.EB3.weight, a=0.01, mode='fan_in',nonlinearity='leaky_relu')
        self.relu3 = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.drop3 = nn.Dropout(p=0.1)

        self.EB4 = nn.Conv1d(7,2, k_EB4,stride=1,padding='same')
        torch.nn.init.kaiming_uniform_(self.EB4.weight, a=0.01, mode='fan_in',nonlinearity='leaky_relu')
        self.relu4 = nn.LeakyReLU(negative_slope=0.01, inplace=False)
        self.drop4 = nn.Dropout(p=0.1)

        self.FB = nn.Conv1d(2,2, k_FB,stride=160, padding = 'valid')
        torch.nn.init.xavier_normal_(self.FB.weight)
        self.sigmoid1 = nn.Sigmoid()
        self.drop5 = nn.Dropout(p=0.0)

        self.sigmoid4 = nn.Sigmoid()

        self.AN1 = nn.Conv1d(2,2, k_DN1,stride=1,padding='same', groups=1)
        torch.nn.init.xavier_normal_(self.AN1.weight)
        self.sigmoidAN1 = nn.Sigmoid()
        self.drop6 = nn.Dropout(p=0.0)

        self.AN2 = nn.Conv1d(2,2, k_DN2,stride=1,padding='same', groups=1)
        torch.nn.init.xavier_normal_(self.AN2.weight)
        self.sigmoidAN2 = nn.Sigmoid()
        self.drop7 = nn.Dropout(p=0.0)

        self.AN3 = nn.Conv1d(2,5, k_DN3,stride=1,padding='same', groups=1)
        torch.nn.init.xavier_normal_(self.AN3.weight)
        self.sigmoidAN3 = nn.Softmax(dim=1)
        self.drop8 = nn.Dropout(p=0.0)

        self.bnorm1 = nn.BatchNorm1d((30))
        self.bnorm2 = nn.BatchNorm1d((15))
        self.bnorm3 = nn.BatchNorm1d((7))
        self.bnorm4 = nn.BatchNorm1d((2))

    def forward(self, x : torch.Tensor, training : int = 0) -> tuple[torch.Tensor, torch.Tensor]:
        """
        The forward pass of the model. The input is passed through the layers of the model and the output is returned.
        The output consists of two tensors: one for the voice activity detection and one for the adversarial network.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor containing the speech data.
        training : int, optional
            Flag denoting whether the model is being trained or tested. The default is 0.

        Returns
        -------
        DB : torch.Tensor
            The output tensor of decoder block of the model.
        AN : torch.Tensor
            The output tensor of the adversarial network block of the model.
        """

        x = x.to(device)
        x = x/2**15

        x = self.EB1(x)
        x = self.relu1(x)
        x = self.bnorm1(x)
        x = self.drop1(x)

        x = self.EB2(x)
        x = self.relu2(x)
        x = self.bnorm2(x)
        x = self.drop2(x)

        x = self.EB3(x)
        x = self.relu3(x)
        x = self.bnorm3(x)
        x = self.drop3(x)

        x = self.EB4(x)
        x = self.relu4(x)
        x = self.bnorm4(x)
        x = self.drop4(x)

        x = self.FB(x)
        x = self.sigmoid1(x)
        x = self.drop5(x)

        DB = self.sigmoid4(x)

        AN = self.AN1(x)
        AN = self.sigmoidAN1(AN)
        AN = self.drop6(AN)

        AN = self.AN2(AN)
        AN = self.sigmoidAN2(AN)
        AN = self.drop7(AN)

        AN = self.AN3(AN)
        AN = self.sigmoidAN3(AN)

        return DB, AN

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
</style>
## Optimizers of the Model

In [ ]:
VAD = VAD_model().to(device)

""" Initialize the optimisers"""

optimizer_EB1 = torch.optim.RMSprop(VAD.EB1.parameters(), lr=learning_rate)
optimizer_EB2 = torch.optim.RMSprop(VAD.EB2.parameters(), lr=learning_rate)
optimizer_EB3 = torch.optim.RMSprop(VAD.EB3.parameters(), lr=learning_rate)
optimizer_EB4 = torch.optim.RMSprop(VAD.EB4.parameters(), lr=learning_rate)

optimizer_FB = torch.optim.RMSprop(VAD.FB.parameters(), lr=learning_rate)

optimizer_DN1 = torch.optim.RMSprop(VAD.AN1.parameters(), lr=learning_rate_DN)
optimizer_DN2 = torch.optim.RMSprop(VAD.AN2.parameters(), lr=learning_rate_DN)
optimizer_DN3 = torch.optim.RMSprop(VAD.AN3.parameters(), lr=learning_rate_DN)


<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
</style>
# Training, Testing Functions

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1.2em;
        color: #465E5B;
    }
</style>
## Utility Functions

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Save model and result to file

In [ ]:
def load_model(path : str) -> None:
    """
    Function for loading a saved model
    """

    global VAD

    VAD.load_state_dict(torch.load(path), strict=False)

    return VAD


def save_results_AUC(res : dict, path : str) -> None:
    """
    Function for saving the dictionary containing information on testing as a pickle file
    """
    with open(path, "wb") as fp:
        pickle.dump(res, fp, protocol=pickle.HIGHEST_PROTOCOL)

def load_results(path : str) -> None:
    """
    Function for loading a saved pickle file
    """
    with open(path, "rb") as input_file:
        return pickle.load(input_file)

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Update learning rate

In [ ]:
def update_learning_rates() -> None:

    """
    Updates the learning rate of all layers"
    """

    global learning_rate
    global learning_rate_DN


    for param_group in optimizer_EB1.param_groups:
        param_group['lr'] = learning_rate
    for param_group in optimizer_EB2.param_groups:
        param_group['lr'] = learning_rate
    for param_group in optimizer_EB3.param_groups:
        param_group['lr'] = learning_rate
    for param_group in optimizer_EB4.param_groups:
        param_group['lr'] = learning_rate
    for param_group in optimizer_FB.param_groups:
        param_group['lr'] = learning_rate
    for param_group in optimizer_DN3.param_groups:
        param_group['lr'] = learning_rate_DN*-1
    for param_group in optimizer_DN1.param_groups:
        param_group['lr'] = learning_rate_DN*-1
    for param_group in optimizer_DN2.param_groups:
        param_group['lr'] = learning_rate_DN*-1

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Calculate loss of VAD

In [ ]:
def calc_loss(true_labels : np.ndarray, predictions : np.ndarray) -> float:
    """
    Calculates the loss from the VAD output, by comparing it to the true labels

    Parameters
    ----------
    true_labels : numpy array
        The true labels of the data
    predictions : numpy array
        The predictions made by the model

    Returns
    -------
    loss : float
        The loss calculated by the loss function, BCELoss
    """
    labels_two_channels = np.zeros((2,len(true_labels[0,:])))
    labels_two_channels = torch.from_numpy(labels_two_channels).to(device)
    index_min = min(len(true_labels[0,:]),len(predictions[0,0,:]))
    labels_two_channels[0,:] = true_labels
    labels_two_channels[1,:] = 1-true_labels
    labels_two_channels = labels_two_channels[:,0:index_min]

    loss = loss_primary(predictions[0,:,0:index_min].T.float(),labels_two_channels[:,:].T.float())
    return loss

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Calculate loss of DN

In [ ]:
def calc_loss_noisetypes(noise_true : np.ndarray, noise_predicted : np.ndarray) -> float:
    """
    Calculates the loss from the discriminative network, by comparing it to the true noise types

    Parameters
    ----------
    noise_true : numpy array
        The true noise types of the data
    noise_predicted : numpy array
        The predictions made by the model

    Returns
    -------
    loss : float
        The loss calculated by the loss function, CrossEntropyLoss
    """
    noise_true = torch.reshape(noise_true, (-1,))
    loss = loss_secondary(noise_predicted.T, noise_true.long())
    return loss

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Back Propagation

In [ ]:
def back_propagation_full(loss : float, t : int) -> None:
    """
    Performs the backward step to calculate gradients, then updates the parameters

    Parameters
    ----------
    loss : float
        The loss calculated by the loss function
    t : int
        The number of forward steps that have been taken (unused for now)

    Returns
    -------
    None
    """

    optimizer_EB1.zero_grad()
    optimizer_EB2.zero_grad()
    optimizer_EB3.zero_grad()
    optimizer_EB4.zero_grad()
    optimizer_FB.zero_grad()
    optimizer_DN1.zero_grad()
    optimizer_DN2.zero_grad()
    optimizer_DN3.zero_grad()
    loss.backward()

    optimizer_EB1.step()
    optimizer_EB2.step()
    optimizer_EB3.step()
    optimizer_EB4.step()

    optimizer_FB.step()

    optimizer_DN1.step()
    optimizer_DN2.step()
    optimizer_DN3.step()

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### After batches

In [ ]:
def after_batches(batch_size: int, accumulated_accuracy: float, loss_acc: float, \
                  batch: int, loss: float, loss_AN: float, X: torch.Tensor, accuracy: float, size: int, loss_L2: float) -> None:
    """
    Prints various information in the console after each backward step

    Parameters
    ----------
    batch_size : int
        The number of forward steps per backward step
    accumulated_accuracy : float
        The accuracy of the model, it is accumulated over the batch size
    loss_acc : float
        The accumulated loss over the batch size
    batch : int
        The number of forward steps that have been taken
    loss : float
        The loss calculated by the loss function
    X : Tensor
        The input data
    accuracy : float
        The accuracy of the model over the batch (unused for now)
    size : int
        The total number of files in the dataset
    loss_L2 : float
        The L2 loss calculated by the loss function

    Returns
    -------
    None
    """
    accumulated_accuracy /= batch_size
    loss, current = loss.item(), batch * len(X)
    loss_acc /= batch_size

    print(f"Current file/Number of files: [{current+1:>5d}/{size:>5d}]")
    print(f"loss VAD: {loss_acc:>7f}")
    print(f"Loss DN: {loss_AN}")
    print(f"Loss L: {loss_L2}")
    print(f"Accuracy of batch: {accumulated_accuracy*100:>4f}")
    print(f"Learning rate: {learning_rate:>4f}")
    print(f"Learning rate DN: {learning_rate_DN:>4f}\n")

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Calculate accuracy

In [ ]:
def calc_accuracy(pred: torch.Tensor, y: torch.Tensor) -> tuple[float, np.ndarray]:
    """
    Finds the predicted labels by comparing the speech and non-speech channels, then calculates the accuracy. Returns both

    Parameters
    ----------
    pred : Tensor
        The predicted output of the model
    y : Tensor
        The true labels of the data

    Returns
    -------
    accuracy : float
        The accuracy of the model
    labs : numpy array
        The predicted labels
    """
    labs = (pred[:,0,0]>pred[:,1,0]).to('cpu').detach().numpy()
    y = y[:,0:len(labs)]
    labs = labs[0:len(y[0,:])]
    accuracy = 1-sum((abs(labs-y[0,:].to('cpu').detach().numpy())))/len(y[0,:].to('cpu').detach().numpy())
    return accuracy, labs

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Make plots

In [ ]:
def make_plots(preds : np.ndarray, X : torch.Tensor, y : torch.Tensor, labs : np.ndarray) -> None:
    """
    Plots the raw waveform, scores of speech and non-speech, VAD predictions and true VAD labels

    Parameters
    ----------
    preds : numpy array
        The predictions made by the model
    X : numpy array
        The input data
    y : numpy array
        The true labels of the data
    labs : numpy array
        The predicted labels
    """
    y = y[0:len(labs)]
    plt.plot(X[0:len(preds[0,:])*160]/max(X)*0.5, label="Amplitude")
    samples_p_label = np.ones((160,))
    # plt.plot(np.kron(preds[0,:], samples_p_label)-0.5,'g')
    # plt.plot(np.kron(preds[1,:], samples_p_label)-0.5,'r')
    plt.plot((np.kron(y, samples_p_label))-0.5,'g', label="True labels")
    plt.plot((np.kron(labs, samples_p_label))*0.5+0.6, 'r',label="Predicted labels")
    plt.plot((np.kron(abs(labs-y), samples_p_label))*0.5-1.4, label="False predicted points")

    new_acc = {1-sum((labs-y!=0))/len(y)}
    plt.ylim([-1.1,1.3])
    plt.title(f"SNR level: {snr} bB - acc: {new_acc}")

    plt.xlabel("Samples")
    plt.legend(loc="best")
    plt.show()
    plt.pause(0.0005)

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Plot ROC and calculate AUC score

In [ ]:
def calculate_AUC_and_plot_ROC() -> None:

  """
  Plot roc curve and calculate auc score.
  """
  y_TP_rate = []
  x_FP_rate = []

  length = len(training_results_AUC["TP"])

  for i in range(length):
    y_rate = (training_results_AUC["TP"][i])/(training_results_AUC["TP"][i] + training_results_AUC["FN"][i])
    x_rate = (training_results_AUC["FP"][i])/(training_results_AUC["FP"][i] + training_results_AUC["TN"][i])

    y_TP_rate.append(y_rate)
    x_FP_rate.append(x_rate)

  auc_score = auc(x_FP_rate, y_TP_rate)

  plt.plot(x_FP_rate, y_TP_rate, marker='o', linestyle='-')  # Plot with markers and line
  plt.fill_between(x_FP_rate, y_TP_rate, alpha=0.2, color='skyblue') # Fill area under the curve


  # Add AUC score text
  plt.text(0.6, 0.2, f"AUC = {auc_score:.4f}", fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("Receiver Operating Characteristic (ROC) Curve")
  plt.grid(True)
  plt.show()




<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-weight: bold;
        font-style: italic;
        font-family: Tw Cen MT;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1em;
        color: #465E5B;
    }
</style>
### Count number of element in confusion matrix

In [ ]:
def calc_pos_neg(labels : np.ndarray, predictions : np.ndarray) -> tuple[int, int, int, int]:
    """
    Calculates the number of true positive, false positives, true negatives and false negatives in the last forward step

    Parameters
    ----------
    labels : numpy array
        The true labels of the data
    predictions : numpy array
        The predictions made by the model

    Returns
    -------
    TP : int
        The number of true positives, the model predicted speech and it was speech
    FP : int
        The number of false positives, the model predicted speech and it was not speech
    TN : int
        The number of true negatives, the model predicted non-speech and it was non-speech
    FN : int
        The number of false negatives, the model predicted non-speech and it was speech
    """
    labels = labels[0:len(predictions)]
    predictions_inv = (predictions-1)*-1;
    labels_inv = (labels-1)*-1;
    TP = sum(predictions[labels[0:len(predictions)]==1])
    FP = sum(predictions[labels[0:len(predictions)]==0])
    FN = sum(predictions_inv[labels[0:len(predictions)]==1])
    TN = sum(predictions_inv[labels_inv[0:len(predictions)]==1])
    return TP, FP, TN, FN

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
</style>
## Training Functions

In [ ]:
def train_loop(train_data_loader : DataLoader, t : int) -> None:
    """Variable initialisations"""
    size = len(train_data_loader.dataset)
    total_acc = 0
    total_loss_DB = 0
    total_loss_AN = 0
    loss_acc = 0
    loss_AN_acc = 0
    accumulated_accuracy = 0
    comb_loss = 0
    last_batch = -1
    concats = concatenates # The number of files to concatenate

    global noises_buffer

    if(len(noises_buffer) == 0):
      noise_1, _ = librosa\
      .load(f"/content/{folder_name}/noises_{snr_noise_str}/babble_3s_{snr_noise}.wav", sr=16000)
      noise_2, _ = librosa\
      .load(f"/content/{folder_name}/noises_{snr_noise_str}/factory_3s_{snr_noise}.wav", sr=16000)

      noises_buffer = [noise_1, noise_2]


    """ Initialises empty tensors for the data to be concatenated"""
    concat_X = torch.empty((1,1,1,0), device = device)
    concat_y = torch.empty((1,0), device = device)
    concat_noise = torch.empty((1,0), device = device)
    bcounter = 0

    for batch, (X, y, noise_true, SNR) in enumerate(train_data_loader):

        y = y.to(device)
        X = X.to(device)


        """Add randomly 0.5s - 2.0s noises to end of tensor """

        number_of_frame = random.randint(50, 200)
        index_of_noise = random.randint(0,1)

        raw_buffer = noises_buffer[index_of_noise][0:number_of_frame*160]

        buffer = np.zeros((1,1,len(raw_buffer)))

        buffer[0,0,:] = np.hstack(raw_buffer)

        buffer_label = np.zeros(number_of_frame)

        buffer = torch.from_numpy(buffer).to(device)
        buffer = torch.unsqueeze(buffer, 0)

        buffer_label = torch.from_numpy(buffer_label).to(device)
        buffer_label = torch.unsqueeze(buffer_label, 0)


        buffer_noise = np.zeros(np.shape(buffer_label)) + index_of_noise
        buffer_noise = torch.from_numpy(buffer_noise).to(device)



        """Creates a tensor of similar size to y
        containing the true labels for noise type"""
        noises = ["N1", "N2", "CL"]
        noise_index = noises.index(''.join(noise_true))

        noise_vector_ini = np.zeros(np.shape(y)) + noise_index
        noise_vector_ini = torch.from_numpy(noise_vector_ini).to(device)


        """Concatenate audio, VAD labels and noise labels"""
        if len(X[0,0,0,:]) != 0:
            concat_X = torch.cat((concat_X, X),3)
            concat_y = torch.cat((concat_y, y),1)
            concat_noise = torch.cat((concat_noise, noise_vector_ini),1)

        """The main training loop.
         Runs after sufficient files are concatenated"""
        if batch >= last_batch + concats:

            bcounter +=1
            last_batch = batch # Counter variable
            """Stores the data in original variable names
             and resets the tensors containing the concatenated files"""
            X = concat_X
            y = concat_y
            noise = concat_noise

            concat_X = torch.empty((1,1,1,0), device = device)
            concat_y = torch.empty((1,0), device = device)
            concat_noise = torch.empty((1,0), device = device)

            concat_X = torch.cat((concat_X, buffer),3)
            concat_y = torch.cat((concat_y, buffer_label),1)
            concat_noise = torch.cat((concat_noise, buffer_noise),1)



            """Forward step"""

            pred_DB, pred_AN = VAD(X[0,:,:,:].float(), training=1)

            loss_DB = calc_loss(y,pred_DB)
            loss_AN = calc_loss_noisetypes(\
                                           noise[:,0:len(pred_AN[0,0,:])], \
                                           pred_AN[0,:,0:len(noise[0,:])])

            l2_penalty = l2_weight * sum([(p**2).sum() for p in VAD.parameters() if p.requires_grad])

            """Variables storing the accumulated losses"""
            total_loss_DB += loss_DB.item() # Accumulated loss over full training epoch
            loss_acc += loss_DB.item()      # Accumulated loss between backward steps
            loss_AN_acc += loss_AN.item()

            comb_loss += (1*loss_DB - AN_weight*loss_AN + l2_penalty)  # Accumulated combined loss of VAD and noise - including the computational graph

            """Calculates the predicted VAD labels and returns the accuracy"""
            accuracy, _ = calc_accuracy(pred_DB.T, y)
            labs = (pred_DB[0,0,:]>pred_DB[0,1,:]).to('cpu').detach().numpy()
            accumulated_accuracy += accuracy
            total_acc += accuracy

            """Backward step"""
            if bcounter == training_batch_size:
                bcounter = 0
                comb_loss = comb_loss/training_batch_size # Finding the mean of the loss
                back_propagation_full(comb_loss, t) # Performs the backpropagation and optimisation step

                after_batches(training_batch_size, accumulated_accuracy, loss_acc, batch, loss_DB, loss_AN_acc, X, accuracy, size, l2_penalty) # Prints information about the latest forward step to the console. Comment to keep the console clean

                """Resets variables"""
                loss_acc = 0
                loss_AN_acc = 0
                accumulated_accuracy = 0
                comb_loss = 0


    training_results_big["training"].append(total_acc/size)
    training_results_big["learning_rate"].append(learning_rate)
    training_results_big["epochs"].append(t)
    training_results_big["loss_DB"].append(total_loss_DB/size)
    training_results_big["loss_AN"].append(total_loss_AN/size)
    save_model_path = f"model_epoch_{t}_{snr_str}_WODB.pth"
    torch.save(VAD.state_dict(), save_model_path)
    return

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
</style>
## Testing Functions

In [ ]:
def test_loop(test_data_loader : DataLoader, t : int) -> None:
    """Variable initialisations"""
    last_batch = -1
    concats = concatenates # The number of files to concatenate

    global noises_buffer

    if(len(noises_buffer) == 0):
      noise_1, _ = librosa\
      .load(f"/content/{folder_name}/noises_{snr_noise_str}/babble_3s_{snr_noise}.wav", sr=16000)
      noise_2, _ = librosa\
      .load(f"/content/{folder_name}/noises_{snr_noise_str}/factory_3s_{snr_noise}.wav", sr=16000)

      noises_buffer = [noise_1, noise_2]


    """ Initialises empty tensors for the data to be concatenated"""
    concat_X = torch.empty((1,1,1,0), device = device)
    concat_y = torch.empty((1,0), device = device)

    for batch, (X, y, noise_true, SNR) in enumerate(test_data_loader):

        if(batch > 30): return;

        y = y.to(device)
        X = X.to(device)


        """Add randomly 0.5s - 2.0s noises to end of tensor """

        number_of_frame = random.randint(50, 200)
        index_of_noise = random.randint(0,1)

        raw_buffer = noises_buffer[index_of_noise][0:number_of_frame*160]

        buffer = np.zeros((1,1,len(raw_buffer)))

        buffer[0,0,:] = np.hstack(raw_buffer)

        buffer_label = np.zeros(number_of_frame)

        buffer = torch.from_numpy(buffer).to(device)
        buffer = torch.unsqueeze(buffer, 0)

        buffer_label = torch.from_numpy(buffer_label).to(device)
        buffer_label = torch.unsqueeze(buffer_label, 0)


        """Concatenate audio, VAD labels and noise labels"""
        if len(X[0,0,0,:]) != 0:
            concat_X = torch.cat((concat_X, X),3)
            concat_y = torch.cat((concat_y, y),1)

        """The main training loop.
         Runs after sufficient files are concatenated"""
        if batch >= last_batch + concats:

            last_batch = batch # Counter variable
            """Stores the data in original variable names
             and resets the tensors containing the concatenated files"""
            X = concat_X
            y = concat_y

            concat_X = torch.empty((1,1,1,0), device = device)
            concat_y = torch.empty((1,0), device = device)

            concat_X = torch.cat((concat_X, buffer),3)
            concat_y = torch.cat((concat_y, buffer_label),1)



            """Forward step"""
            start_time = time.time()
            pred_DB, pred_AN = VAD(X[0,:,:,:].float())
            end_time = time.time()

            """Time to excute model"""
            execution_time = (end_time - start_time) * 1000


            """Calculates the predicted VAD labels and returns the accuracy"""
            accuracy, _ = calc_accuracy(pred_DB.T, y)
            labs = (pred_DB[0,0,:]>pred_DB[0,1,:]).to('cpu').detach().numpy()

            print(f"Accuracy : {accuracy}")

            npX = X[0,0,0,:].to('cpu').detach().numpy()
            npy = y[0,:].to('cpu').detach().numpy()
            preds_np = pred_DB[0,:,:].to('cpu').detach().numpy()

            ipd.display(ipd.Audio(npX, rate=16000))

            print('Make plot')
            make_plots(preds_np, npX, npy, labs)

            """ROC plot"""
            ROC_samples = 51
            TP_acc = np.zeros((ROC_samples,))
            FP_acc = np.zeros((ROC_samples,))
            TN_acc = np.zeros((ROC_samples,))
            FN_acc = np.zeros((ROC_samples,))
            accumulated_acc = np.zeros((ROC_samples,))

            for index, (threshold) in enumerate(np.linspace(0,1,ROC_samples)):

                labs = (pred_DB[0,0,:]>threshold).to('cpu').detach().numpy()
                npy=y[0,:].to('cpu').detach().numpy()
                npy = npy[0:len(labs)]

                """ Calculates  the number of true positives, false positive etc."""
                TP, FP, TN, FN = calc_pos_neg(npy, labs)

                """Each index corresponds to a threshold value"""
                TP_acc[index] += TP
                FP_acc[index] += FP
                TN_acc[index] += TN
                FN_acc[index] += FN


            total_samples = FP_acc + TN_acc + TP_acc + FN_acc

            training_results_AUC[f"TP"] = (TP_acc/total_samples)
            training_results_AUC[f"FP"] = (FP_acc/total_samples)
            training_results_AUC[f"TN"] = (TN_acc/total_samples)
            training_results_AUC[f"FN"] = (FN_acc/total_samples)


            print(f"Time to execute model: {execution_time}")
            calculate_AUC_and_plot_ROC()

    return

<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1.25em;
        color: #465E5B;
    }
</style>
# Evaluation of the Model
Running functions

In [ ]:
def run_train(t : int) -> None:

    """Training process

    Parameters
    ----------
    t : int
        The number of epochs that have been run

    Returns
    -------
    None
    """


    global learning_rate, learning_rate_DN, LR_factor

    print(f"Epoch {t+1}\n--------------TRAIN-----------------")
    VAD.train()
    dataset_train = TIMIT_train()
    train_data_loader = DataLoader(dataset_train, batch_size=1, shuffle=True)
    update_learning_rates()
    train_loop(train_data_loader, t) # The main training loop

    """Updates the learning rate after each epoch"""
    learning_rate *= LR_factor
    learning_rate_DN *= LR_factor

In [ ]:
def run_test(t : int) -> None:

    """Testing process

    Parameters
    ----------
    t : int
        The number of epochs that have been run

    Returns
    -------
    None
    """

    print(f"Epoch {t+1}\n--------------TEST-----------------")
    dataset_test = TIMIT_test()
    test_data_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)
    test_loop(test_data_loader, t)



<style>
    h1 {
        color: #FBA2D0;
        font-weight: bolder;
        font-family: Aptos;
    }
    h2 {
        color: #62825D;
        font-weight: bolder;
        font-family: Aptos;
    }
    h3 {
        color: #FA9189;
        font-style: italic;
        font-family: Roboto;
    }
    * {
        font-family: Tw Cen MT;
        font-size: 1.25em;
        color: #465E5B;
    }
</style>
## Main method

In [ ]:
pretrained = True
continue_train = False

path_to_model = ""

if(pretrained):
  load_model(path_to_model)
  run_test(19)

else:
  for t in range(training_epochs):
      run_train(t)
      run_test(t)